In [ ]:

import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors


movies_path = "D:/project/movies_metadata.csv"  
ratings_path = "D:/project/ratings.csv"         


movies = pd.read_csv(movies_path, encoding='utf-8')
ratings = pd.read_csv(ratings_path, encoding='utf-8')


movies = movies[movies['id'].apply(lambda x: str(x).isdigit())]
movies['movieId'] = movies['id'].astype(int)

print("Movies columns:", movies.columns.tolist())
print("Ratings columns:", ratings.columns.tolist())


def get_content_recommendations(title, top_n=5):
    if 'genres' not in movies.columns or 'title' not in movies.columns:
        return ["Movies data missing required columns."]
    

    tfidf = TfidfVectorizer(stop_words='english')
    tfidf_matrix = tfidf.fit_transform(movies['genres'].fillna(''))
    
    
    model_knn = NearestNeighbors(metric='cosine', algorithm='brute')
    model_knn.fit(tfidf_matrix)
    
    indices = pd.Series(movies.index, index=movies['title']).drop_duplicates()

    if title not in indices:
        return ["Movie not found in dataset."]

    idx = indices[title]
    distances, indices_neighbors = model_knn.kneighbors(tfidf_matrix[idx], n_neighbors=top_n+1)

    recommended_indices = indices_neighbors.flatten()[1:]  # exclude the movie itself
    return movies['title'].iloc[recommended_indices].tolist()


def get_collaborative_recommendations(title, top_n=5):
    if 'movieId' not in movies.columns or 'movieId' not in ratings.columns:
        return ["Missing 'movieId' in movies or ratings data."]
    if 'rating' not in ratings.columns or 'userId' not in ratings.columns:
        return ["Missing 'userId' or 'rating' in ratings data."]
    try:
        user_movie_matrix = ratings.merge(movies[['movieId', 'title']], on='movieId')
    except Exception:
        return ["Error merging ratings and movies data."]
    matrix = user_movie_matrix.pivot_table(index='userId', columns='title', values='rating').fillna(0)

    if title not in matrix.columns:
        return ["Movie not rated by users. Try another."]

    try:
        model_knn = NearestNeighbors(metric='cosine', algorithm='brute')
        model_knn.fit(matrix.T.values)
        query_index = list(matrix.columns).index(title)
        distances, indices_neighbors = model_knn.kneighbors([matrix.T.values[query_index]], n_neighbors=top_n+1)
        recommended = [matrix.columns[i] for i in indices_neighbors.flatten()][1:]
        return recommended
    except Exception as e:
        return [f"Collaborative filtering failed: {e}"]


print("\nExample movie titles (first 10):")
print(movies['title'].dropna().unique()[:10])

movie_input = input("\nEnter a movie title (exactly as shown above): ").strip()
method = input("Choose recommendation method ('content' or 'collaborative'): ").strip().lower()

if method == 'content':
    recommendations = get_content_recommendations(movie_input)
elif method == 'collaborative':
    recommendations = get_collaborative_recommendations(movie_input)
else:
    recommendations = ["Invalid recommendation method chosen."]

print("\nTop 5 recommendations:")
for i, rec in enumerate(recommendations, 1):
    print(f"{i}. {rec}")


C:\Users\91636\AppData\Local\Temp\ipykernel_28960\2269423410.py:13: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies = pd.read_csv(movies_path, encoding='utf-8')


Movies columns: ['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id', 'imdb_id', 'original_language', 'original_title', 'overview', 'popularity', 'poster_path', 'production_companies', 'production_countries', 'release_date', 'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title', 'video', 'vote_average', 'vote_count', 'movieId']
Ratings columns: ['userId', 'movieId', 'rating', 'timestamp']

Example movie titles (first 10):
['Toy Story' 'Jumanji' 'Grumpier Old Men' 'Waiting to Exhale'
 'Father of the Bride Part II' 'Heat' 'Sabrina' 'Tom and Huck'
 'Sudden Death' 'GoldenEye']



Enter a movie title (exactly as shown above):  Toy Story
Choose recommendation method ('content' or 'collaborative'):  content



Top 5 recommendations:
1. Tom and Jerry: Shiver Me Whiskers
2. Kronk's New Groove
3. Santa Claus and the Magic Drum
4. The Prince and the Pauper
5. An All Dogs Christmas Carol
